##  Chunking - Optimizing Vector Database Data Preparation

- Character/Token Based Chunking
- Recursive Character/Token Based Chunking
- Semantic Chunking
- Cluster Semantic Chunking
- LLM Semantic Chunking


### The Chunking Evaluation Repo

In [1]:
%%capture
!pip install git+https://github.com/brandonstarxel/chunking_evaluation.git

In [2]:
# Main Chunking Functions
from chunking_evaluation.chunking import (
    ClusterSemanticChunker,
    LLMSemanticChunker,
    FixedTokenChunker,
    RecursiveTokenChunker,
    KamradtModifiedChunker
)
# Additional Dependencies
import tiktoken
from chromadb.utils import embedding_functions
from chunking_evaluation.utils import openai_token_count
import os

Pride and Prejudice by Jane Austen, available for free from Project Gutenberg, will be used. It consists of 476 pages of text or 175,651 tokens.

In [4]:

with open("./pride_and_prejudice.txt", 'r', encoding='utf-8') as file:
        document = file.read()

print("First 1000 Characters: ", document[:1000])

First 1000 Characters:  ﻿The Project Gutenberg eBook of Pride and Prejudice
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Pride and Prejudice

Author: Jane Austen

Release date: June 1, 1998 [eBook #1342]
                Most recently updated: June 17, 2024

Language: English

Credits: Chuck Greif and the Online Distributed Proofreading Team at http://www.pgdp.net (This file was produced from images available at The Internet Archive)


*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***




                            [Illustration:

                             GEORGE A

#### **Helper Function for Analyzing Chunking!**

- **Display Chunk Count**: _The function prints the length of the provided chunks list (i.e., the number of chunks)._
- **Examine Specific Chunks**: _It prints the 200th and 201st chunks (indices 199 and 200)._

- **Overlap Analysis**: _It identifies overlapping text between the 200th and 201st chunks, checked in two modes._
    + **Character-Based** (use_tokens=False): _Searches for a common substring between the two chunks._
    + **Token-Based** (use_tokens=True): _Uses the tiktoken library to tokenize the text and checks for token overlap._

In [6]:
def analyze_chunks(chunks, use_tokens=False):
    # Print the chunks of interest
    print("\nNumber of Chunks:", len(chunks))
    print("\n", "="*50, "200th Chunk", "="*50,"\n", chunks[199])
    print("\n", "="*50, "201st Chunk", "="*50,"\n", chunks[200])

    chunk1, chunk2 = chunks[199], chunks[200]

    if use_tokens:
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens1 = encoding.encode(chunk1)
        tokens2 = encoding.encode(chunk2)

        # Find overlapping tokens
        for i in range(len(tokens1), 0, -1):
            if tokens1[-i:] == tokens2[:i]:
                overlap = encoding.decode(tokens1[-i:])
                print("\n", "="*50, f"\nOverlapping text ({i} tokens):", overlap)
                return
        print("\nNo token overlap found")
    else:
        # Find overlapping characters
        for i in range(min(len(chunk1), len(chunk2)), 0, -1):
            if chunk1[-i:] == chunk2[:i]:
                print("\n", "="*50, f"\nOverlapping text ({i} chars):", chunk1[-i:])
                return
        print("\nNo character overlap found")

### <font color='orange'>**Character Text Splitting**</font>

_The simplest form of chunking would be simply counting some number of characters and splitting at that count._

In [7]:
def chunk_text(document, chunk_size, overlap):
    chunks = []
    stride = chunk_size - overlap
    current_idx = 0

    while current_idx < len(document):
        # Take chunk_size characters starting from current_idx
        chunk = document[current_idx:current_idx + chunk_size]
        if not chunk:  # Break if we're out of text
            break
        chunks.append(chunk)
        current_idx += stride  # Move forward by stride

    return chunks

Chunk size of `400` Characters, `no overlap`

In [8]:
character_chunks = chunk_text(document, chunk_size=400, overlap=0)

analyze_chunks(character_chunks)


Number of Chunks: 1871

 ================================================== 200th Chunk ================================================== 
 ty to their aunt, and
to a milliner’s shop just over the way. The two youngest of the family,
Catherine and Lydia, were particularly frequent in these attentions:
their minds were more vacant than their sisters’, and when nothing
better offered, a walk to Meryton was necessary to amuse their morning
hours and furnish conversation for the evening; and, however bare of
news the country in general mi

 ================================================== 201st Chunk ================================================== 
 ght be, they always contrived to learn
some from their aunt. At present, indeed, they were well supplied both
with news and happiness by the recent arrival of a militia regiment in
the neighbourhood; it was to remain the whole winter, and Meryton was
the head-quarters.

Their visits to Mrs. Philips were now productive of the most interes

Chunk size of `800` Characters, `400` overlap

In [9]:
character_overlap_chunks = chunk_text(document, chunk_size=800, overlap=400)

analyze_chunks(character_overlap_chunks)


Number of Chunks: 1871

 ================================================== 200th Chunk ================================================== 
 ty to their aunt, and
to a milliner’s shop just over the way. The two youngest of the family,
Catherine and Lydia, were particularly frequent in these attentions:
their minds were more vacant than their sisters’, and when nothing
better offered, a walk to Meryton was necessary to amuse their morning
hours and furnish conversation for the evening; and, however bare of
news the country in general might be, they always contrived to learn
some from their aunt. At present, indeed, they were well supplied both
with news and happiness by the recent arrival of a militia regiment in
the neighbourhood; it was to remain the whole winter, and Meryton was
the head-quarters.

Their visits to Mrs. Philips were now productive of the most interesting
intelligence. Every day added something to their knowled

 ================================================== 201s

### <font color='orange'>**Token Text Splitting**</font>

But language models (the end users of chunked text usually) don't operate at the character level. Instead they use tokens, or common sequences of characters that represent frequent words, word pieces, and subwords. For example, the word 'hamburger' when ran through GPT-4's tokenizer is split into the tokens ['h', 'amburger']. Common words like 'the' or 'and' are typically single tokens.

This means character-based splitting isn't ideal because:

    1. A 500-character chunk might contain anywhere from 100-500 tokens depending on the text
    2. Different languages and character sets encode to very different numbers of tokens
    3. We might hit token limits in our LLM without realizing it

A good visualizer of tokenization is available [on OpenAI's platform](https://platform.openai.com/tokenizer)

Tokenizers like 'cl100k_base' implement Byte-Pair Encoding (BPE) - a compression algorithm that creates a vocabulary by iteratively merging the most frequent pairs of bytes or characters. The '100k' refers to its vocab size, determining the balance between compression and representation granularity.

When talking to a language model, the first step is tokenizing the text so that it can be processed by the underlying neural network. The LLM outputs tokens which are decoded back into words.

In [10]:
import tiktoken

# Loading cl100k_base tokenizer
encoder = tiktoken.get_encoding("cl100k_base")

# Text Example
text = "hamburger"
tokens = encoder.encode(text)

print("Tokens:", tokens)

Tokens: [71, 47775]


In [11]:
for i in range(len(tokens)):
    print(f"Token {i+1}:", encoder.decode([tokens[i]]))

print("Full Decoding: ", encoder.decode(tokens))

Token 1: h
Token 2: amburger
Full Decoding:  hamburger


#### **Helper Function for Counting Tokens**

In [12]:
def count_tokens(text, model="cl100k_base"):
    """Count tokens in a text string using tiktoken"""
    encoder = tiktoken.get_encoding(model)
    return print(f"Number of tokens: {len(encoder.encode(text))}")

Chunk Size of `400` Tokens, `0 Overlap`

In [13]:
fixed_token_chunker = FixedTokenChunker(
    chunk_size=400,
    chunk_overlap=0,
    encoding_name="cl100k_base"
)

token_chunks = fixed_token_chunker.split_text(document)

analyze_chunks(token_chunks, use_tokens=True)


Number of Chunks: 440

 ================================================== 200th Chunk ================================================== 
  fortunate as to meet Miss Bennet. The
subject was pursued no further, and the gentlemen soon afterwards went
away.




[Illustration:

“At Church”
]




CHAPTER XXXI.


[Illustration]

Colonel Fitzwilliam’s manners were very much admired at the Parsonage,
and the ladies all felt that he must add considerably to the pleasure of
their engagements at Rosings. It was some days, however, before they
received any invitation thither, for while there were visitors in the
house they could not be necessary; and it was not till Easter-day,
almost a week after the gentlemen’s arrival, that they were honoured by
such an attention, and then they were merely asked on leaving church to
come there in the evening. For the last week they had seen very little
of either Lady Catherine or her daughter. Colonel Fitzwilliam had called
at the Parsonage more than once dur

In [14]:
count_tokens(token_chunks[0])

Number of tokens: 400


Chunk Size of `400` Tokens, `200` Overlap

In [15]:
fixed_token_chunker = FixedTokenChunker(
    chunk_size=400,
    chunk_overlap=200,
    encoding_name="cl100k_base"
)

token_overlap_chunks = fixed_token_chunker.split_text(document)

analyze_chunks(token_overlap_chunks, use_tokens=True)


Number of Chunks: 878

 ================================================== 200th Chunk ================================================== 
  I _heard_ nothing of his going away when I
was at Netherfield. I hope your plans in favour of the ----shire will
not be affected by his being in the neighbourhood.”

“Oh no--it is not for _me_ to be driven away by Mr. Darcy. If _he_
wishes to avoid seeing _me_ he must go. We are not on friendly terms,
and it always gives me pain to meet him, but I have no reason for
avoiding _him_ but what I might proclaim to all the world--a sense of
very great ill-usage, and most painful regrets at his being what he is.
His father, Miss Bennet, the late Mr. Darcy, was one of the best men
that ever breathed, and the truest friend I ever had; and I can never be
in company with this Mr. Darcy without being grieved to the soul by a
thousand tender recollections. His behaviour to myself has been
scandalous; but I verily believe I could forgive him anything and
every

### <font color='orange'>**Recursive Character Text Splitter**</font>

When we write, we naturally separate text into paragraphs, sentences, and other logical units. The recursive character text splitter tries to intelligently split text by looking for natural separators in order, while respecting a maximum character length.

First, it makes a complete pass over the entire document using paragraph breaks (\n\n), creating an initial set of chunks. Then for any chunks that exceed the size limit, it recursively processes them using progressively smaller separators:

    1. First tries to split on paragraph breaks (\n\n)
    2. If chunks are still too big, tries line breaks (\n)
    3. Then sentence boundaries (., ?, !)
    4. Then words ( )
    5. Finally, if no other separators work, splits on individual characters ("")

This way, the splitter preserves as much natural structure as possible - only drilling down to smaller separators when necessary to meet the size limit. A chunk that's already small enough stays intact, while larger chunks get progressively broken down until they fit.

**Chunk Size of `800` Characters, `0` Overlap**

In [16]:
recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=800,  # Character Length
    chunk_overlap=0,  # Overlap
    length_function=len,  # Character length with len()
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_character_chunks = recursive_character_chunker.split_text(document)
analyze_chunks(recursive_character_chunks, use_tokens=False)


Number of Chunks: 1270

 ================================================== 200th Chunk ================================================== 
 When tea was over Mr. Hurst reminded his sister-in-law of the
card-table--but in vain. She had obtained private intelligence that Mr.
Darcy did not wish for cards, and Mr. Hurst soon found even his open
petition rejected. She assured him that no one intended to play, and the
silence of the whole party on the subject seemed to justify her. Mr.
Hurst had, therefore, nothing to do but to stretch himself on one of the
sofas and go to sleep. Darcy took up a book. Miss Bingley did the same;
and Mrs. Hurst, principally occupied in playing with her bracelets and
rings, joined now and then in her brother’s conversation with Miss
Bennet.

 ================================================== 201st Chunk ================================================== 
 Miss Bingley’s attention was quite as much engaged in watching Mr.
Darcy’s progress through _his_ book, 

In [17]:
len(recursive_character_chunks[199]) # Chunk 200

635

This means we don't get exact splits - a chunk might be 550 characters long because that's where a paragraph or sentence naturally ends, rather than forcing the full 800 character limit. The chunker prioritizes maintaining these natural text boundaries over hitting the exact maximum size.